In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

Generating train split:   0%|          | 0/11615290 [00:00<?, ? examples/s]

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [4]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
print('importé tokenizer')
# Embeddear las letras de las canciones
tokenized_messages = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])
print('tokenicé')
tokenized_messages = tokenized_messages.apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])
print('padding')
# Create a new DataFrame with 'track_name' and the tokenized lyrics
df_tokenized = pd.DataFrame({'vector': tokenized_messages, 'name': df['n']})

df_tokenized.head()

importé tokenizer


KeyboardInterrupt: 

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
db = lancedb.connect("./messages.lancedb")
# Creamos una tabla en la base de datos
db.create_table("tabla", df_tokenized)
db["tabla"].head()

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
(db["tabla"].search()
    .where("LENGTH(texto) > 1000")
    .select(["name", "vector"])
    .limit(5)
    .to_pandas())

-Query en SQL equivalente: select texto from tabla where len(texto) >1000 limit 5


-Explicacion: Me basé mucho en la clase, solamente que como son textos quería ver si había oraciones que juntaran más de 1000 porque ese fue el tokenaizer o el tamaño que usé primero y tardaba demasiado en correr, por ello, quería ver si de hecho sí había que fueran mayores a 1000

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [ ]:
mi_mensaje = 'I think therefore i am i AM'
mi_mensaje_tokenized = tokenizer(mi_mensaje)["input_ids"]

print(len(mi_mensaje_tokenized))

if len(mi_mensaje_tokenized) < 300:
    mi_mensaje_tokenized = mi_mensaje_tokenized + [0] * (300 - len(mi_mensaje_tokenized))
else:
    mi_mensaje_tokenized = mi_mensaje_tokenized[:300]

df_query = (db["tabla"].search(mi_mensaje_tokenized)
        .metric("cosine") # Puede ser L2 o cosine
        .select(["vector"])
        .limit(1)
        .to_pandas())

query_tokenized = df_query['vector'].values[0]

mensaje_parecido =  tokenizer.decode(query_tokenized, skip_special_tokens=True)